# Ex9: 3次元の座標変換

以下のノートブックを実行して，合成変換をデザインしてください．

フォルダに
- mesh.py
- suzanne.obj
をアップロードしてから課題を進めてください．


## データの読み込みと補助関数

viewer周りにメッシュの描画関数まで入っています．

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import mpl_toolkits.mplot3d.art3d as art3d

from mesh import *

mesh = loadOBJ('suzanne.obj')
mesh.computeFaceNormals()

V = mesh.V
F = mesh.F
N_f = mesh.N_f


def compare_PQ(P, Q, F, N_f):
    num_faces = len(F)

    fig = plt.figure(figsize=(15, 15))

    viewer = MeshViewer(fig)

    viewer.addMesh(P, F, [0.6, 0.6, 1.0],  edgecolor=[0.2, 0.2, 1.0], alpha=1.0, linewidth=1.0)
    viewer.addMesh(Q, F, [1.0, 0.6, 0.6], edgecolor=[1.0, 0.2, 0.2], alpha=1.0, linewidth=1.0)
    viewer.setView(elev=20, azim=30)
    viewer.show()


## 平行移動

点$\mathbf{p}$をそれぞれ$t_x, t_y, t_z$だけ移動させる変換は次の式で表せます．

$\mathbf{q} = \mathbf{T}(t_x, t_y t_z) \mathbf{p}$

$\left[\begin{matrix}X\\Y\\Z\\1\end{matrix}\right] = \left[\begin{matrix}1 & 0 & 0 & t_{x}\\0 & 1 & 0 & t_{y}\\0 & 0 & 1 & t_{z}\\0 & 0 & 0 & 1\end{matrix}\right] \left[\begin{matrix}x\\y\\z\\1\end{matrix}\right]$

以下のパラメータtx, ty, tzを動かして，移動結果を確認しましょう．

In [ ]:
# @title 平行移動パラメータ { run: "auto" }
def translate(tx, ty, tz):
    T = np.array([[1, 0, 0, tx],
                  [0, 1, 0, ty],
                  [0, 0, 1, tz],
                  [0, 0, 0, 1]], dtype=float)
    return T


tx = 1  # @param {type: "slider", min: -4.0, max: 4.0, step: 0.01}
ty = 1  # @param {type: "slider", min: -4.0, max: 4.0, step: 0.01}
tz = 1  # @param {type: "slider", min: -4.0, max: 4.0, step: 0.01}

T = translate(tx, ty, tz)

P = np.c_[V, np.ones(len(V))]

Q = P  @ T.T
Q /= Q[:, 3].reshape(-1, 1)

compare_PQ(P, Q, F, N_f)


## 拡大・縮小

点$\mathbf{p}$をそれぞれ$s_x, s_y, s_z$倍に拡大・縮小させる変換は次の式で表せます．

$\mathbf{q} = \mathbf{S}(s_x, s_y s_z) \mathbf{p}$

$\left[\begin{matrix}X\\Y\\Z\\1\end{matrix}\right] = \left[\begin{matrix}s_{x} & 0 & 0 & 0\\0 & s_{y} & 0 & 0\\0 & 0 & s_{z} & 0\\0 & 0 & 0 & 1\end{matrix}\right] \left[\begin{matrix}x\\y\\z\\1\end{matrix}\right]$

以下のパラメータsx, sy, szを動かして，拡大・縮小結果を確認しましょう．

In [ ]:
# @title 拡大縮小パラメータ { run: "auto" }
def scale(sx, sy, sz):
    S = np.array([[sx, 0, 0, 0],
                  [0, sy, 0, 0],
                  [0, 0, sz, 0],
                  [0, 0, 0, 1]], dtype=float)
    return S


sx = 2  # @param {type: "slider", min: 0.0, max: 2.0, step: 0.01}
sy = 1  # @param {type: "slider", min: 0.0, max: 2.0, step: 0.01}
sz = 1  # @param {type: "slider", min: 0.0, max: 2.0, step: 0.01}

S = scale(sx, sy, sz)
T = translate(2.5, 0, 0)

A = T @ S

P = np.c_[V, np.ones(len(V))]

Q = P @ A.T
Q /= Q[:, 3].reshape(-1, 1)

compare_PQ(P, Q, F, N_f)


## 回転

点$\mathbf{p}$をx軸周り，y軸周り，z軸周りに回転させる行列は以下の形になっています．

### ①x軸周りの回転

$\mathbf{q} = \mathbf{R}_x(\theta) \mathbf{p}$

$\left[\begin{matrix}X\\Y\\Z\\1\end{matrix}\right] = \left[\begin{matrix}1 & 0 & 0 & 0\\0 & \cos{\left (\theta \right )} & - \sin{\left (\theta \right )} & 0\\0 & \sin{\left (\theta \right )} & \cos{\left (\theta \right )} & 0\\0 & 0 & 0 & 1\end{matrix}\right] \left[\begin{matrix}x\\y\\z\\1\end{matrix}\right]$

### ②y軸周りの回転

$\mathbf{q} = \mathbf{R}_y(\theta) \mathbf{p}$

$\left[\begin{matrix}X\\Y\\Z\\1\end{matrix}\right] = \left[\begin{matrix}\cos{\left (\theta \right )} & 0 & \sin{\left (\theta \right )} & 0\\0 & 1 & 0 & 0\\- \sin{\left (\theta \right )} & 0 & \cos{\left (\theta \right )} & 0\\0 & 0 & 0 & 1\end{matrix}\right] \left[\begin{matrix}x\\y\\z\\1\end{matrix}\right]$

### ③z軸周りの回転
$\mathbf{q} = \mathbf{R}_z(\theta) \mathbf{p}$

$\left[\begin{matrix}X\\Y\\Z\\1\end{matrix}\right] = \left[\begin{matrix}\cos{\left (\theta \right )} & - \sin{\left (\theta \right )} & 0 & 0\\\sin{\left (\theta \right )} & \cos{\left (\theta \right )} & 0 & 0\\0 & 0 & 1 & 0\\0 & 0 & 0 & 1\end{matrix}\right] \left[\begin{matrix}x\\y\\z\\1\end{matrix}\right]$

以下のパラメータtheta_x, theta_y, theta_zを動かして回転結果を確認してみましょう．


In [ ]:
# @title 回転パラメータ { run: "auto" }
def rotate_x(theta):
    t = np.pi * theta / 180
    c, s = np.cos(t), np.sin(t)
    Rx = np.array([[1, 0,  0, 0],
                   [0, c, -s, 0],
                   [0, s,  c, 0],
                   [0, 0,  0, 1]], dtype=float)
    return Rx


def rotate_y(theta):
    t = np.pi * theta / 180
    c, s = np.cos(t), np.sin(t)
    Ry = np.array([[c, 0, s, 0],
                   [0, 1, 0, 0],
                   [-s, 0, c, 0],
                   [0, 0, 0, 1]], dtype=float)
    return Ry


def rotate_z(theta):
    t = np.pi * theta / 180
    c, s = np.cos(t), np.sin(t)
    Rz = np.array([[c, -s, 0, 0],
                   [s, c, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]], dtype=float)
    return Rz


theta_x = -40.68  # @param {type: "slider", min: -180, max: 180, step: 0.01}
theta_y = 0.0  # @param {type: "slider", min: -180, max: 180, step: 0.01}
theta_z = 0.0  # @param {type: "slider", min: -180, max: 180, step: 0.01}

R_x = rotate_x(theta_x)
R_y = rotate_y(theta_y)
R_z = rotate_z(theta_z)
R = R_z @ R_y @ R_x

T = translate(2.5, 0, 0)

A = T @ R

P = np.c_[V, np.ones(len(V))]

Q = P @ A.T
Q /= Q[:, 3].reshape(-1, 1)

compare_PQ(P, Q, F, N_f)


## Work: 合成変換のデザイン

行列演算を組み合わせて合成変換を定義できます．<br>
例えば，上の回転変換の例では，

1. x軸周りの回転: $\mathbf{R}_x(\theta_x)$
2. y軸周りの回転: $\mathbf{R}_y(\theta_y)$
3. z軸周りの回転: $\mathbf{R}_z(\theta_z)$
4. 平行移動: $\mathbf{T}(t_x, t_y, t_z)$

を組み合わせて，元のモデルに回転結果が被らないように，$\mathbf{T}(t_x, t_y, t_z)$でずらしてモデルを表示しています．

これを順に組み合わせてできる変換は，

$\mathbf{q} = \mathbf{T}(t_x, t_y, t_z) \mathbf{R}_z(\theta_z) \mathbf{R}_y(\theta_y)  \mathbf{R}_x(\theta_x) \mathbf{p}$

のように書けます．<br>
（行列が順番にかかって変換されているのを想像してください）

コードだと，以下の計算がそれに該当します．


```
# 回転行列の計算
R = R_z @ R_y @ R_x

# 全体の座標変換行列の計算
A = T @ R
```

scale, translate, rotate_x, rotate_y, rotate_zの関数を活用して変換の要素行列を作成し，合成変換の形でAを作ってください．



In [ ]:
# Work: 合成変換をデザインしてください．（以下を修正）
A = np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]], dtype=float)

A = T @ R @ A

P = np.c_[V, np.ones(len(V))]

Q = P @ A.T
Q /= Q[:, 3].reshape(-1, 1)

compare_PQ(P, Q, F, N_f)
